# 05tools_2b: Understanding Model Performance and Fairness with the Language Interpretability Tool (LIT)

The [Language Interpretability Tool (LIT)](https://pair-code.github.io/lit/) helps understand a model behavior across a wide range of inputs.  In this notebook the 05 series models will be evaluated with the LIT tool.  While it might sound like LIT is specific to language models,it actually works well with tabular models!

This notebook will show how to connect the tool to the model for predictions (Vertex AI Endpoint) and load a dataset from BigQuery.

### Prerequisites:
-  At least 1 of the notebooks in this series [05, 05a-05i]

---
## Installing LIT
The first time using LIT in the notebook environment will require an install of the python package lit-nlp:
- From a Terminal: `pip install lit-nlp -U -q`
- From a Cell of User-Managed Workbench: `!pip install lit-nlp -U -q`
- From a Cell of a Managed Workbench: `!pip install --user lit-nlp -U -q`

---
## Environment Setup

inputs:

In [130]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [131]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '05tools_2b'
SERIES = '05'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [132]:
from google.cloud import aiplatform
from google.cloud import bigquery

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

from lit_nlp.api import dataset as lit_dataset
from lit_nlp.api import types as lit_types
from lit_nlp.api import model as lit_model
from lit_nlp.components import minimal_targeted_counterfactuals
from lit_nlp import notebook

import numpy as np

clients:

In [133]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project=PROJECT_ID)

parameters:

In [134]:
BUCKET = PROJECT_ID
DIR = f"temp/{NOTEBOOK}"

environment:

In [135]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get Vertex AI Endpoint And Deployed Model

In [136]:
endpoints = aiplatform.Endpoint.list(filter = f"display_name={SERIES}_{DATANAME}")
endpoint = endpoints[0]

In [137]:
endpoint.display_name

'05_fraud'

In [138]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

In [139]:
model.display_name

'05_fraud'

In [140]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_05_fraud@2'

In [141]:
model.uri

'gs://statmike-mlops-349915/fraud/models/05/20220815200908/model'

## Get Data for Model Exploration
Retrive the test data for this series:

In [142]:
test = bq.query(query = f"SELECT * EXCEPT({VAR_TARGET}), {VAR_TARGET} FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' ORDER BY {VAR_TARGET} DESC").to_dataframe()
test = test[test.columns[~test.columns.isin(VAR_OMIT.split()+['splits'])]]
test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,148074,-2.219219,0.727831,-5.458230,5.924850,3.932464,-3.085984,-1.677870,0.865075,-3.177260,...,0.417472,-0.817343,-0.028752,0.025723,-0.825835,-0.013089,0.413291,-0.131387,0.0,1
1,129668,0.753356,2.284988,-5.164492,3.831112,-0.073622,-1.316596,-1.855495,0.831079,-1.567514,...,0.382007,0.033958,0.187697,0.358433,-0.488934,-0.258802,0.296145,-0.047174,2.0,1
2,56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,-2.451251,...,0.794372,0.270471,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1
3,146998,-2.064240,2.629739,-0.748406,0.694992,0.418178,1.392520,-1.697801,-6.333065,1.724184,...,6.215514,-1.276909,0.459861,-1.051685,0.209178,-0.319859,0.015434,-0.050117,8.0,1
4,78725,-4.312479,1.886476,-2.338634,-0.475243,-1.185444,-2.112079,-2.122793,0.272565,0.290273,...,0.550541,-0.067870,-1.114692,0.269069,-0.020572,-0.963489,-0.918888,0.001454,60.0,1


In [143]:
test.shape

(28522, 31)

In [144]:
val = bq.query(query = f"SELECT * EXCEPT({VAR_TARGET}), {VAR_TARGET} FROM {DATANAME}.{DATANAME}_prepped WHERE splits='VALIDATE' ORDER BY {VAR_TARGET} DESC").to_dataframe()
val = val[val.columns[~val.columns.isin(VAR_OMIT.split()+['splits'])]]
val.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,85285,-7.030308,3.421991,-9.525072,5.270891,-4.024630,-2.865682,-6.989195,3.791551,-4.622730,...,1.103398,-0.541855,0.036943,-0.355519,0.353634,1.042458,1.359516,-0.272188,0.0,1
1,142961,0.457845,1.373769,-0.488926,2.805351,1.777386,0.100492,1.295016,-0.135857,-1.695822,...,0.105593,0.371014,0.051105,0.401524,-0.724766,-0.202881,0.092124,0.094956,0.0,1
2,149640,0.754316,2.379822,-5.137274,3.818392,0.043203,-1.285451,-1.766684,0.756711,-1.765722,...,0.397058,0.141165,0.171985,0.394274,-0.444642,-0.263189,0.304703,-0.044362,2.0,1
3,92092,-1.108478,3.448953,-6.216972,3.021052,-0.529901,-2.551375,-2.001743,1.092432,-0.836098,...,0.825951,1.144170,0.208559,-0.295497,-0.690232,-0.364749,0.229327,0.208830,18.0,1
4,94362,-26.457745,16.497472,-30.177317,8.904157,-17.892600,-1.227904,-31.197329,-11.438920,-9.462573,...,-8.755698,3.460893,0.896538,0.254836,-0.738097,-0.966564,-7.263482,-1.324884,1.0,1


## Setting Up LIT
At a minimimum, LIT requires a Dataset and Model specification following the requires [Type System](https://github.com/PAIR-code/lit/wiki/api.md#type-system):
- Dataset
    - Use the Type System to describe and format the examples as flat dictionaries using a custom Class
- Model
    - A custom class made up of Python functions that return inputs, outputs, and predictions following the Type System

Common Inputs:

In [145]:
VOCABS = {f'{VAR_TARGET}': ['Not Fraud', 'Fraud']}
VAR_SPECS = test.dtypes.apply(lambda x: x.name).to_dict()

### LIT Dataset
The class specification here for `FraudDataset` is built in a way that automate the specification by leveraging the knowledge that all the fields except the target, stored in `VAR_TARGET` above, are all numeric. The class also uses the commmon inputs defined in the previous cell/section. 

Define Class for Dataset:

In [146]:
class FraudDataset(lit_dataset.Dataset):
    
    def __init__(self, ds):
        records = ds.to_dict(orient='records')
        self._examples = []
        for rec in records:
            rec[f'{VAR_TARGET}'] = VOCABS[f'{VAR_TARGET}'][rec[f'{VAR_TARGET}']]
            self._examples.append(rec)
            
    def spec(self):
        specs = VAR_SPECS.copy()
        for s in specs:
            if s == VAR_TARGET: specs[s] = lit_types.CategoryLabel(vocab = VOCABS[VAR_TARGET])
            else: specs[s] = lit_types.Scalar()        
        return specs

Test Class:

In [147]:
test_ds = FraudDataset(test)
val_ds = FraudDataset(val)

In [148]:
len(val_ds.examples), len(test_ds.examples)

(28461, 28522)

In [149]:
#test_ds.examples[0], test_ds.spec()

In [150]:
#val_ds.examples[0], val_ds.spec()

### LIT Model
The class specification here for `FraudModel` is built to use the Vertex AI endpoint as a prediction service via the `endpoint.predict` method.

In [151]:
class FraudModel(lit_model.Model):
    
    def __init__(self, endpoint):
        self.model = endpoint

    def max_minibatch_size(self):
        return 2000        
        
    def predict_minibatch(self, inputs):
        instances = [json_format.ParseDict({key:value for key, value in example.items() if key != VAR_TARGET}, Value()) for example in inputs]
        predictions = endpoint.predict(instances = instances).predictions
        
        # unpack the batch
        for pred in predictions:
            output = {'predictions': pred}
            yield output
        #return [{'predictions': out} for out in predictions]
        
    def input_spec(self):
        specs = VAR_SPECS.copy()
        specs.pop(VAR_TARGET, None) # remove the target being predicted
        for s in specs:
            specs[s] = lit_types.Scalar()
        return specs
    
    def output_spec(self):
        return {
            'predictions': lit_types.MulticlassPreds(
                parent = f'{VAR_TARGET}', vocab = VOCABS[VAR_TARGET]
            )
        }

In [156]:
widget = notebook.LitWidget(
    models = {'classification': FraudModel(endpoint)},
    datasets = {'test': FraudDataset(test[0:4000]), 'val': FraudDataset(val[0:4000])},
    generators = {'Minimal Targeted Counterfactuals': minimal_targeted_counterfactuals.TabularMTC()},
    height = 800
)

### LIT Widget

In [ ]:
widget.render()

In [158]:
widget.stop()

---
## Example Screenshot

<img src="architectures/notebooks/05tools_2_screenshots/lit.png">